In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import sklearn.preprocessing as p
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import wrangle as w
import seaborn as sns
import acquire as a
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
train, validate, test = w.wrangle_zillow()

AttributeError: 'Series' object has no attribute 'total_sqft'

In [ ]:
train.shape, validate.shape, test.shape

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
# remove .0 from zip_code and total_sqft

train.zip_code.convert_dtypes(int)

Begin exploration